In [1]:
from brainlit.utils import upload, session, upload_benchmarking
from pathlib import Path
import napari

c:\users\shrey\anaconda3\envs\ndd-windows\lib\site-packages\python_jsonschema_objects\__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Uploading Benchmarking Images from local data locations .
This notebook demonstrates uploading the benchmarking data and associated `.swc` segment files.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload. If uploading .tif files that aren't very large or where only one resolution is necessary, set num_res to 1. 
 
The below paths lead to sample data in my local drive (will be updated to reflect download from S3). Alter the below path definitions to point to your own local file locations.

In [2]:
source = Path("C:/Users/shrey/Downloads/1").as_posix()
#source = Path("C:/Users/shrey/Downloads/benchmarking_datasets").as_posix()
source_segments = Path("C:/Users/shrey/Downloads/1/consensus-swcs").as_posix()
dest = "s3://open-neurodata/benchmarking_data/1"
dest_segments = "s3://open-neurodata/benchmarking_data/1"
num_res = 1

## 2) Upload the image data (.tif)

In [3]:
upload_benchmarking.upload_volumes(source, dest, num_res)

Preparing files.
Starting upload.




Finished layer index 0, took 12.833412170410156 seconds


If the upload fails with the error: `timed out on a chunk on layer index 0. moving on to the next step of pipeline`, re-run the `upload_volumes` function but with the `continue_upload` parameter, which takes `layer index` (the layer index said in the error message) and `image index` (the last succesful image that uploaded).  

For example, if the output failed after image 19, then run 
`upload_benchmarking.upload_volumes(source, dest, num_res, continue_upload = (0, 19))`. Repeat this till all of the upload is complete.


## 3) Upload the segmentation data (.swc)

In [3]:
upload_benchmarking.upload_segments(source, dest_segments, num_res)

(0, 0, 0)


Uploading:   0%|                                              | 0/1 [00:00<?, ?it/s]

Uploading: 100%|██████████████████████████████████████| 1/1 [00:00<00:00, 21.06it/s]


# Downloading

In [3]:
%%capture
sess = session.NeuroglancerSession(url=dest, url_segments=dest_segments, mip=0)  # create session object object
img, bounds, vertices = sess.pull_vertex_list(1, list(range(1,187)), 0, expand=True)  # get image containing some data
#labels = sess.create_tubes(1, [330,300,100], radius=1)  # generate labels via tube segmentation
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

OutOfBoundsError: Bbox([0, 0, 0],[330, 330, 100], dtype=int32) did not fully contain the specified bounding box Bbox([0, 0, 300],[330, 330, 400], dtype=int32).

In [11]:
# get image and center point
ngl_sess = session.NeuroglancerSession(mip = 0, url = dest, url_segments=dest_segments)
img, bbox, vox = ngl_sess.pull_voxel(seg_id = 1, v_id = 1, radius = 0)
print(f"\n\nDownloaded volume is of shape {img.shape}, with total intensity {sum(sum(sum(img)))}.")

Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 40.10it/s]


OutOfBoundsError: Bbox([0, 0, 0],[330, 330, 100], dtype=int32) did not fully contain the specified bounding box Bbox([224, 53, 344],[225, 54, 345], dtype=int32).

In [9]:
files_ordered, paths_bin, vox_size, img_size, origin = upload_benchmarking.get_volume_info(source, num_res)

In [10]:
x_range, y_range, z_range = upload_benchmarking.get_data_ranges(['000'], img_size)
z_range

['000']
000


[0, 100]

In [6]:
import numpy as np
sess.cv_segments.skeleton.get(1).vertices[1]
#sess.cv.info

Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 10.67it/s]


array([73902.875, 17479.256, 34378.67 ], dtype=float32)

In [7]:
voxel = sess._get_voxel(1,1)
from cloudvolume.lib import Bbox
#bounds = Bbox(voxel, voxel)
bounds = Bbox([0,0,0],[329,329,99])
seed = bounds.to_list()
shape = 0 * 3
bounds = Bbox(np.subtract(seed[:3], shape), np.add(np.add(seed[3:], shape), 1))
#img = sess.pull_bounds_img(bounds)
bounds
#sess.pull_vertex_list(1, list(range(1,187)), 0, expand = True)
sess.pull_bounds_img.check_iterable_nonnegative(bounds)
#img = sess.cv.download(Bbox(bounds[:3], bounds[3:]), mip=self.mip)
#img = sess.cv.download(Bbox(bounds[:3], bounds[3:]), mip=self.mip)


Downloading: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  6.87it/s]


TypeError: 'Bbox' object is not subscriptable

In [11]:
origin

(0, 0, 0)

In [2]:
import boto3
import os
HOME = Path("C:/Users/shrey")
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('open-neurodata')
objects = my_bucket.objects.filter(Prefix='benchmarking_data/tif_files/')
for obj in objects:
    path, filename = os.path.split(obj.key)
    my_bucket.download_file(obj.key, filename)
#boto3.set_stream_logger('botocore', level='DEBUG')
#s3.download_file('open-neurodata', 'benchmarking_data', 'tif_files')
#'test_1-gfp.tif'

NoCredentialsError: Unable to locate credentials